## Pre-process image stack

For a final stack like `fst-concat-rgb-small.tif`, we had to do some pre-processing.

 - a. Concatenate all individual raw stacks. Cellpose wants >5 cells for training.
 - b. Make the combined stack (a) into an RGB stack.
 - c. Reduce the size of that raw stack (downsample). Cellpose wants DAPI cell bodies to be about 10 pixels (check this number). This stack is saved as `fst-concat-rgb-small.tif`.
 - d. Export the stack (c) to a folder with one slice per file. Cellpose trains on slices, not stacks. Make sure each slice is iteself RGB.
 - e. Use cellpose interface to train the model on the individual slice images in the folder (d)
 
Once we have trained a model, we can apply that model to a 3D RGB stack like `fst-concat-rgb-small.tif` that was created in (c).

Cellpose will save a `_seg.npy` file whith the output of the model including a mask of (in this case) DAPI cell bodies.
    

## Online Tutorials

Here, I am doing a classic textbook mask analysis by making a DAPI 'ring' mask (with erosion and dilation) and counting pixels in the Oligo channel that are within that mask.

#### **Note, I only discovered this after the entire process was done!!!**

https://scikit-image.org/docs/stable/auto_examples/applications/plot_fluorescence_nuclear_envelope.html#sphx-glr-auto-examples-applications-plot-fluorescence-nuclear-envelope-py

A very good online respurce with lots of image processing tutorials in Python and visualized with Napari.

See: https://haesleinhuepf.github.io/BioImageAnalysisNotebooks/31_graphical_user_interfaces/napari.html


## Import Python packages

In [ ]:
import os
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.measure import regionprops, regionprops_table
import scipy

import seaborn as sns
import plotly.express as px

from IPython.display import display, HTML

import napari
import napari_layer_table  # our custom plugin (installed from source)

## Load the 3D RGB stack that we applied the model to.

In [ ]:
# what the model was trained on
tifPath = '/Users/cudmore/Dropbox/data/whistler/data-oct-10/cellpose-model/fst-concat-small-slices/fst-concat-rgb-small.tif'
# label 69 is 100% hit
# label 86

# model was applided to this
#tifPath = '/Users/cudmore/Dropbox/data/whistler/data-oct-10/Morphine/Morphine-rgb-merged-small.tif'

filename = os.path.split(tifPath)[1]

imageData = imread(tifPath)

# print out the spatial dimensions of the image
print(filename)
print('imageData:', imageData.shape, imageData.dtype)

## Split the RGB stack into individual color channels.

- Oligo/red is channel 0
- DAPI/green is channel 1

In [ ]:
# oligo channel
redChannelIdx = 0
imageData_red = imageData[:,:,:,redChannelIdx]  # (slice, x, y)
print('imageData_red:', imageData_red.shape, imageData_red.dtype)

# dapi channel
greenChannelIdx = 1
imageData_green = imageData[:,:,:,greenChannelIdx]  # (slice, x, y)
print('imageData_green:', imageData_green.shape, imageData_green.dtype)

## Open a napari viewer

In [ ]:
# Create an empty viewer
viewer = napari.Viewer()

# Remove all layers to start from scratch
# for l in viewer.layers:
#     viewer.layers.remove(l)

Add original 3D RGB stack

In [ ]:
# Add a new layer containing the rgb image
rgbLayer = viewer.add_image(imageData, name=filename)

Add green and red channels

In [ ]:
redLayer = viewer.add_image(imageData_red)
redLayer.colormap = 'red'
redLayer.contrast_limits = (0, 128)

greenLayer = viewer.add_image(imageData_green)
greenLayer.colormap = 'green'
greenLayer.contrast_limits = (0, 128)


Define a function to run the napari-layer-table plugin

In [ ]:
# run napari-layer-table
def runPlugin(viewer, layer, onAddCallback=None):

    # create the plugin
    ltp = napari_layer_table.LayerTablePlugin(viewer, oneLayer=layer, onAddCallback=onAddCallback)
    #ltp.signalDataChanged.connect(on_user_edit_points2)

    # show
    area = 'right'
    name = layer.name
    _dockWidget = viewer.window.add_dock_widget(ltp, 
                        area=area, name=name)

    return ltp

#runPlugin(viewer, dapi_dilated_mask_layer)

## Threshold the red channel to generate a binary mask

This is completely independ from anything we have done with cellpose.
 1. Gaussian blur
 2. Threshold with Otsu


In [ ]:
from skimage.filters import gaussian
from skimage.filters import threshold_otsu

# blur
imageData_red_blurred = gaussian(imageData_red, sigma=1)

# Add to napari
#viewer.add_image(imageData_red_blurred)

threshold = threshold_otsu(imageData_red_blurred)
print(f'{filename} otsu threshold: {threshold}')

red_binary_image = imageData_red_blurred > threshold  # [True, False]

#print('unique in red_binary_image', np.unique(red_binary_image))

#### Add `red_binary_image` to viewer

In [ ]:
# Add a new labels layer containing the `red_binary_image`
binary_layer = viewer.add_labels(red_binary_image, name="red binary image")

## Plot the histogram of the blurred red channel

Vertical dotted line is otsu threshold

In [ ]:
fig = px.histogram(x=imageData_red_blurred.flatten(), log_y=True)
fig.update_layout(xaxis_title="Image Intensity")

fig.add_vline(x=threshold, line_width=3, line_dash="dash", line_color="black")

fig.show()

## Make some functions to load cellpose results.

We need to
 1. Load the cellpose `_seg.npy` file.
 2. Extract some info from it, like the 3D DAPI/green masks.

In [ ]:
def getSeg_npy(tifPath):
    """Given an original tif path, get the cellpose '_seg.npy' file.
    """
    _path, _file = os.path.split(tifPath)
    segFile = os.path.splitext(_file)[0] + '_seg.npy'
    segPath = os.path.join(_path, segFile)
    return segPath

def loadSegMask_npy(tifPath):
    """Given a tif file, load masks from cellpose _seg.npy file.
    """
    segPath = getSeg_npy(tifPath)

    dat = np.load(segPath, allow_pickle=True).item()
    masks = dat['masks']
    return masks

## Load the results of running the cellpose model on our 3D RGB stack

We trained the `cyto` model on the dapi channel (check exactly what I did)

In [ ]:
cellpose_dapi_mask = loadSegMask_npy(tifPath)
print('cellpose_dapi_mask:', cellpose_dapi_mask.shape, cellpose_dapi_mask.dtype)

numLabels = len(np.unique(cellpose_dapi_mask))
print('num labels in cellpose_dapi_mask:', numLabels)
print('remember, label 0 is always background')

Add `cellpose_dapi_mask` to Napari

In [ ]:
cellpose_dapi_mask_layer = viewer.add_labels(cellpose_dapi_mask, name="cellpose_dapi_mask")

## Adjust the cellpose dapi mask with dilation and erosion
 
For each label in `cellpose_dapi_mask`:
  - Make a dilated version
  - Make an eroded version
  - Make a ring as different (xor) between dilated and eroded
  
Using the DAPI mask, for the red mask, calculate
  - a. Number of pixels (size) of DAPI mask
  - b. Number of pixels in Oligo mask contained in the DAPI mask
  - c. Percent of pixels in Oligo channel that are in mask (b/a)

In [ ]:
numMasks = np.max(cellpose_dapi_mask)

dapi_final_mask = np.zeros_like(cellpose_dapi_mask)  # dapi mask after dilation
print('making dapi_dilated_mask', dapi_final_mask.shape, dapi_final_mask.dtype)

listOfDict = []  # convert to pandas dataframe at end

dilationIterations = 1  # was 3

# numMasks+1 to include the last mask
for maskIdx in range(numMasks+1):
    if maskIdx == 0:
        # background
        continue
    
    _oneMask = cellpose_dapi_mask == maskIdx  # (46, 196, 196)
    #print('_oneMask:', type(_oneMask), _oneMask.shape, _oneMask.dtype)

    # dilate the mask
    _dilatedMask = scipy.ndimage.binary_dilation(_oneMask, iterations=dilationIterations)
    _erodedMask = scipy.ndimage.binary_erosion(_oneMask, iterations=dilationIterations)
    #print('  dilatedMask:', type(dilatedMask), dilatedMask.shape, dilatedMask.dtype, np.sum(dilatedMask))

    # make a ring
    #dilatedMask = dilatedMask ^ _oneMask
    finalMask = _dilatedMask ^ _erodedMask  # carrot (^) is xor

    # the number of pixels in the dilated/eroded dapi mask
    finalMaskCount = np.count_nonzero(finalMask)

    # oligo red mask pixels in the (dilated/eroded) dapi mask
    redImageMask = np.where(finalMask==True, red_binary_image, 0)  # 0 is fill value
    #print('  redImageMask:', type(redImageMask), redImageMask.shape, redImageMask.dtype, np.sum(redImageMask))

    # like cellpose_dapi_mask but after dilation
    finalMaskLabel = finalMask.copy().astype(np.int64)
    #print('1 ', dilatedMaskLabel.dtype, np.max(dilatedMaskLabel))
    # +1 so colors are different from cellpose_dapi_mask
    finalMaskLabel[finalMaskLabel>0] = maskIdx + 1   
    #print('  2 ', dilatedMaskLabel.dtype, np.max(dilatedMaskLabel))
    dapi_final_mask = dapi_final_mask + finalMaskLabel
    #print('  dapi_dilated_mask:', dapi_dilated_mask.shape, np.sum(dapi_dilated_mask))
    
    redImageMaskPercent = np.sum(redImageMask) / finalMaskCount * 100
    
    oneDict = {
        'label': maskIdx,
        'redImageMaskSum': np.sum(redImageMask),  # sum of red mask in dilated dapi mask
        'finalMaskCount': finalMaskCount,  # num pixels in dilated mask
        'redImageMaskPercent': redImageMaskPercent,  # fraction of pixels in red mask in dilated mask
    }
    listOfDict.append(oneDict)

    # no, labelIdx corresponds to napari label
    #    NO: labelIdx here is (label + 1) in napari
    #    NO: labelIdx 52 corresponds to 53 in napari
    # in napari, label (83, 88) are 100% positive dapi+oligo
    
dfMaster = pd.DataFrame(listOfDict)

# these are the dapi masks with a good amount of red
redImageMaskPercent_threshold = 25
print('\nredImageMaskPercent_threshold:', redImageMaskPercent_threshold)
print('Using this threshold, we have the following DAPI mask candidates')
print('. e.g. the ones with a lot of Oligo red)')
print(dfMaster[dfMaster['redImageMaskPercent']>=redImageMaskPercent_threshold])


#### Add final dapi mask to Napari (`dapi_final_mask`)

In [ ]:
# my dapi mask after dilation
# view this with threshold red channel
print('dapi_final_mask:', dapi_final_mask.shape, np.sum(dapi_final_mask))
dapi_dilated_mask_layer = viewer.add_labels(dapi_final_mask, name="dapi_final_mask")

## Inspect DAPI mask with red channel to note oligo nuclei

Using Napari ...

`cellpose_dapi_mask` and `imageData_red` to note the DAPI/nuclei you would mark that are Oligo nuclei. Basically, there is red around the DAPI/nucleus labelling.

#### File: fst-concat-rgb-small.tif

Manual Labels:

 - 19 (z 3)
 - 53 (z 20)
 - 69 (z 30)
 - 73 (z 33)
 - 86 (z 39)
 - 105 (z 42)

TODO: Add these to dfMaster

Notes:
    label 53 shows small % Oligo because cellpose DAPI mask is HUGE
    
#### File: Morphine-rgb-merged-small.tif

Manual Labels:
 - 24 (z 6)
 - 53 (z 17) ???
 - 42 (z 20) ???
 - 83 (z 29)
 - 88 (z 34)
 - 113 (z 41) huge DAPI mask

In [ ]:
# make a column to score DAPI mask labels as (Unknown, Oligo)
dapiScoring = ['Unknown'] * (numMasks)

# File: fst-concat-rgb-small.tif
if filename == 'fst-concat-rgb-small.tif':
    dapiScoring[19-1] = 'Oligo'
    dapiScoring[53-1] = 'Oligo'
    dapiScoring[69-1] = 'Oligo'
    dapiScoring[73-1] = 'Oligo'
    dapiScoring[86-1] = 'Oligo'
    dapiScoring[105-1] = 'Oligo'

# File: Morphine-rgb-merged-small.tif 
if filename == 'Morphine-rgb-merged-small.tif':
    dapiScoring[24-1] = 'Oligo'
    dapiScoring[53-1] = 'Oligo'
    dapiScoring[42-1] = 'Oligo'
    dapiScoring[83-1] = 'Oligo'
    dapiScoring[88-1] = 'Oligo'
    dapiScoring[113-1] = 'Oligo'

#dapiScoring.insert(0, "Unknown")

#print(dapiScoring)

dfMaster['dapiScoring'] = dapiScoring

display(dfMaster[dfMaster['dapiScoring']=='Oligo'])

In [ ]:
#myLabels = [19, 53, 69,73, 86,105]

fig = px.scatter(dfMaster, x='label', y='redImageMaskPercent',
                    size='finalMaskCount',
                    color='dapiScoring',
                     title=filename,
                    template='plotly_dark')
# for aLabel in myLabels:
#     fig.add_vline(x=aLabel, line_width=2, line_dash='dash', line_color='black')
fig.add_hline(y=redImageMaskPercent_threshold,
                      line_width=2, line_dash='dash', line_color='red')

fig.update_layout(xaxis_title="DAPI Mask (label)")
fig.update_layout(yaxis_title="Oligo (%) redImageMaskPercent")

print(filename)
print('size is the size of the dapi mask "finalMaskCount"')
print('"Unknown" and "Oligo" are manual scoring, looking for Oligo around DAPI')

fig.show()

## Generate statistics for each DAPI mask

Using `cellpose_dapi_mask`, we will get properties like:

 - label of each mask (1, 2, 3, ...)
 - centroid (z, y, x) of each mask
 - area (how big or small is each mask)
    
TODO: Look up other properties

In [ ]:
print('cellpose_dapi_mask:', \
          cellpose_dapi_mask.shape, \
          cellpose_dapi_mask.dtype, \
          'min:', np.min(cellpose_dapi_mask),
          'max:', np.max(cellpose_dapi_mask),
     )

# regionprop does not return row label 0 (backgroun)
_properties = ['label', 'centroid', 'area']  # 'solidity' gives convex-hull error
props_dict = regionprops_table(cellpose_dapi_mask, properties=_properties)

print('props_dict.keys:', props_dict.keys())

# df from props
dfProps = pd.DataFrame(props_dict)

# rename some columns
dfProps = dfProps.rename(columns={'centroid-0': 'z', 'centroid-1': 'y', 'centroid-2': 'x'})

#print(dfProps)

# statistics : [skimage.measure._regionprops.RegionProperties]
_regionprops = regionprops(cellpose_dapi_mask)

# add to dfMaster
#print('dfMaster:', dfMaster)

dfMaster['x'] = dfProps['x'].tolist()
dfMaster['y'] = dfProps['y'].tolist()
dfMaster['z'] = dfProps['z'].tolist()
dfMaster['area'] = dfProps['area'].tolist()

display(dfMaster)


## Look at DAPI mask size and percent Oligo

This reveals a false-negative because DAPI mask was huge, making Oligo percent small.

Note: we still have a few false-positives. We will handle them by scoring each "candidate" Oligo/DAPI manually.
    

In [ ]:
## Compare our DAPI mask size to 
fig = px.scatter(dfMaster, x='finalMaskCount', y='area',
                     size='redImageMaskPercent',
                    color='dapiScoring',
                     title=filename,
                    template='plotly_dark')

fig.update_layout(xaxis_title="DAPI Mask (pixels) finalMaskCount")
fig.update_layout(yaxis_title="DAPI Mask (area) area")

print('Size is "redImageMaskPercent"')

fig.show()

## Make a Napari points layer from the region props of cellpose mask

Using `cellpose_dapi_mask`.

This is so we can browse using napari-layer-table plugin.

In [ ]:
# make a points layer from centroid
_regionprops = regionprops(cellpose_dapi_mask)

# add centroid to napari
points = [s.centroid for s in _regionprops]  # point[i] is a tuple of (z, y, x)
points.insert(0, [0, 0, 0])

#print('points:', len(points), points[0])

# assign napari face_color
face_color = ['blue'] * len(points)
oligoLabelList = dfMaster[dfMaster['dapiScoring']=='Oligo']['label'].tolist()
#print('oligoLabelList:', oligoLabelList)
for _oligoLabel in oligoLabelList:
    face_color[_oligoLabel] = 'red'

# add a 'redImageMaskPercent' column as a property
_percentList = dfMaster['redImageMaskPercent'].tolist()
_percentList.insert(0, 0)

_finalMaskCount = dfMaster['finalMaskCount'].tolist()
_finalMaskCount.insert(0, 0)

properties = {
    'redImageMaskPercent': _percentList,
    'finalMaskCount': _finalMaskCount,
}

# add points to viewer
label_layer_points = viewer.add_points(points,
                                       face_color=face_color,
                                       symbol='cross',
                                       size=5,
                                      properties=properties)

#print(type(label_layer_points.properties))

#runPlugin(viewer, label_layer_points)

In [ ]:
runPlugin(viewer, label_layer_points)

## I started this process like this

Trying to draw a vertical line here. To the right are DAPI/Oligo candidates.

In [ ]:
# plot with plotly
fig = px.histogram(dfMaster, x='redImageMaskPercent', log_y=True)
fig.show()